### This file is dedicated to scrape and download data from domain.com.au

Yuecheng Wang Aug 30, modify from ./scripts/scrape.py

In [1]:
import os
import re
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
import requests
import time
import json
import numpy as np

In [2]:
# Load the postcodes from your JSON file
with open('../../data/raw/suburb_to_postcodes.json', 'r') as f:
    suburb_to_postcodes = json.load(f)

# Flatten the list of postcodes into a single list
postcodes = [postcode for sublist in suburb_to_postcodes.values() for postcode in sublist]
print(postcodes)

[3206, 3182, 3143, 3054, 3053, 3052, 3000, 3001, 3004, 3066, 3067, 3008, 3002, 3183, 3184, 3065, 3068, 3031, 3051, 3003, 3027, 3181, 3121, 3205, 3141, 3006, 3182, 3142, 3103, 3104, 3130, 3128, 3129, 3105, 3106, 3107, 3108, 3125, 3147, 3124, 3146, 3126, 3125, 3127, 3129, 3148, 3166, 3167, 3168, 3169, 3109, 3111, 3123, 3150, 3170, 3122, 3101, 3102, 3149, 3131, 3132, 3133, 3151, 3195, 3196, 3197, 3204, 3165, 3184, 3186, 3187, 3163, 3161, 3162, 3192, 3185, 3188, 3193, 3144, 3145, 3194, 3195, 3163, 3166, 3018, 3025, 3028, 3011, 3012, 3033, 3034, 3337, 3338, 3015, 3021, 3219, 3022, 3023, 3020, 3037, 3030, 3029, 3012, 3016, 3013, 3011, 3047, 3064, 3056, 3058, 3044, 3064, 3057, 3040, 3041, 3043, 3033, 3038, 3036, 3042, 3039, 3032, 3046, 3060, 3044, 3058, 3429, 3055, 3083, 3088, 3099, 3095, 3094, 3078, 3081, 3084, 3079, 3082, 3076, 3070, 3072, 3073, 3074, 3075, 3071, 3757, 3153, 3155, 3136, 3140, 3156, 3134, 3135, 3178, 3152, 3176, 3179, 3160, 3806, 3977, 3164, 3767, 3175, 3802, 3804, 3805, 397

In [3]:
# Constants
BASE_URL = "https://www.domain.com.au"
property_metadata = []

Block below is code for scrape CURRENT data from domain.com.au \
I've commented this so that we keep consistent with data scraped 20 Sept 2024 \
for scrape current data, uncomment below code and run all. \
for data used in this project, goto, run all. \
project data are stored on in manual/all_properties_combined.csv which uploaded to our github repo

In [4]:
# # Define the property types and number of bedrooms for each category
# categories = {
#     "apartment": [1, 2, 3],  # Flats with 1, 2, 3 bedrooms
#     "house": [2, 3, 4]       # Houses with 2, 3, 4 bedrooms
# }

# # Define the number of parts to split the postcodes into
# num_parts = 10
# split_postcodes = np.array_split(postcodes, num_parts)

# # Create a temporary folder for storing the part files
# output_relative_dir = '../../data/raw/domain/'
# temp_folder = os.path.join(output_relative_dir, 'temp_parts')
# os.makedirs(temp_folder, exist_ok=True)

# temp_files = []

# BASE_URL = "https://www.domain.com.au"

# # Iterate through each part of the postcodes
# for i, part_postcodes in enumerate(split_postcodes):
#     temp_file = os.path.join(temp_folder, f'property_part_{i+1}.csv')
    
#     # Check if the file already exists
#     if os.path.exists(temp_file):
#         print(f"File for Part {i+1} already exists. Skipping processing for this part.")
#         temp_files.append(temp_file)
#         continue

#     part_property_metadata = []

#     # Process each postcode in the current part
#     for postcode in part_postcodes:
#         N_PAGES = range(1, 2)

#         # Collect links for each property type and number of bedrooms
#         for ptype, bedrooms_list in categories.items():
#             for bedrooms in bedrooms_list:
#                 url_links = []
#                 for page in N_PAGES:
#                     url = BASE_URL + f"/rent/?ptype={ptype}&bedrooms={bedrooms}&postcode={postcode}&sort=dateupdated-desc&page={page}"
#                     print(f"Visiting {url}")

#                     # Fetch and process the page
#                     try:
#                         bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "html.parser")
#                     except Exception as e:
#                         print(f"Failed to load page {url}: {e}")
#                         continue

#                     result_section = bs_object.find("ul", {"data-testid": "results"})
#                     if not result_section:
#                         print(f"No results found on page {url}")
#                         continue

#                     index_links = result_section.findAll("a", href=re.compile(f"{BASE_URL}/*"))
#                     for link in index_links:
#                         if link.get('class') and 'address' in link['class']:
#                             url_links.append(link['href'])

#                 # Now, scrape property details from each collected link
#                 for property_url in tqdm(url_links, desc=f"Processing Part {i+1}, Postcode {postcode}"):
#                     try:
#                         bs_object = BeautifulSoup(urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "html.parser")
#                         time.sleep(0.5)  # Delay to avoid rate limiting
#                     except Exception as e:
#                         print(f"Failed to load property page {property_url}: {e}")
#                         continue

#                     try:
#                         # Extract property details
#                         address = bs_object.find("h1", {"class": "css-164r41r"}).text if bs_object.find("h1", {"class": "css-164r41r"}) else "N/A"
#                         cost_text = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text if bs_object.find("div", {"data-testid": "listing-details__summary-title"}) else "N/A"

#                         # Extract property type
#                         property_type = ptype.capitalize()

#                         # Extract number of bedrooms
#                         bathrooms = "N/A"
#                         rooms_section = bs_object.find("div", {"data-testid": "property-features"})
#                         if rooms_section:
#                             rooms = rooms_section.findAll("span", {"data-testid": "property-features-text-container"})
#                             for feature in rooms:
#                                 text = feature.text.lower()
#                                 if 'bath' in text:
#                                     bathrooms = re.findall(r'\d+', text)[0] if re.findall(r'\d+', text) else "N/A"

#                         # Attempt to find the link that contains the coordinates
#                         try:
#                             href = bs_object.find("a", {"target": "_blank", 'rel': "noopener noreferrer"})
#                             href = href.attrs['href'] if href else ""
#                             coordinates = [float(coord) for coord in re.findall(r'destination=([-\s,\d\.]+)', href)[0].split(',')] if href else [None, None]
#                             latitude, longitude = coordinates if len(coordinates) == 2 else (None, None)
#                         except (AttributeError, IndexError, ValueError):
#                             latitude, longitude = None, None

#                         # Extract closest government secondary school information
#                         gov_school_name, gov_school_distance = "N/A", "N/A"
#                         school_section = bs_object.find("div", {"data-testid": "listing-details__school-catchment"})
#                         if school_section:
#                             gov_section = school_section.find("h3", string="Government School Catchment")
#                             if gov_section:
#                                 gov_secondary = gov_section.find_next("ul")
#                                 if gov_secondary:
#                                     gov_schools = gov_secondary.find_all("li", {"data-testid": "fe-co-school-catchment-school"})
#                                     for school in gov_schools:
#                                         levels = school.find_all("li", {"data-testid": "fe-co-school-catchment-tags"})
#                                         if any('7-12' in level.text or '7-10' in level.text or '10-12' in level.text for level in levels):
#                                             gov_school_name = school.find("h4").text
#                                             gov_school_distance = school.find("p").text
#                                             break

#                         # Extract Neighbourhood Insights age data
#                         age_under_20, age_20_39, age_40_59, age_60_plus = "N/A", "N/A", "N/A", "N/A"
#                         neighbourhood_section = bs_object.find("div", {"data-testid": "listing-details-neighbourhood-insights-container"})
#                         if neighbourhood_section:
#                             age_table = neighbourhood_section.find("table", {"class": "css-8atqhb"})
#                             if age_table:
#                                 age_rows = age_table.find_all("tr", {"data-testid": "neighbourhood-insights__age-brackets-row"})
#                                 for row in age_rows:
#                                     age_label = row.find("td", {"data-testid": "neighbourhood-insights__label"}).text
#                                     age_value = row.find("div", {"data-testid": "bar-value"}).text
#                                     if "under 20" in age_label.lower():
#                                         age_under_20 = age_value
#                                     elif "20 - 39" in age_label.lower():
#                                         age_20_39 = age_value
#                                     elif "40 - 59" in age_label.lower():
#                                         age_40_59 = age_value
#                                     elif "60+" in age_label.lower():
#                                         age_60_plus = age_value

#                         # Append the flattened property data to the list
#                         part_property_metadata.append({
#                             'Address': address,
#                             'Cost': cost_text,
#                             'Property Type': property_type,  # House or Apartment
#                             'Bedrooms': bedrooms,
#                             'Bathrooms': bathrooms,
#                             'Latitude': latitude,  # New Latitude field as float
#                             'Longitude': longitude,  # New Longitude field as float
#                             'Closest Gov Secondary School': gov_school_name,
#                             'Gov Secondary Distance': gov_school_distance,
#                             'Age under 20': age_under_20,   
#                             'Age 20-39': age_20_39,        
#                             'Age 40-59': age_40_59,        
#                             'Age 60+': age_60_plus,         
#                             'Postcode': postcode
#                         })

#                     except AttributeError as e:
#                         print(f"Issue with {property_url}: {e}")

#     # Save the current part to a temporary file
#     if part_property_metadata:
#         df = pd.DataFrame(part_property_metadata)
#         df.to_csv(temp_file, index=False)
#         temp_files.append(temp_file)
#         print(f"Completed processing Part {i+1}/{num_parts}. Data saved to {temp_file}")

# # Combine all parts into one large file
# combined_df = pd.concat([pd.read_csv(file) for file in temp_files])

# # Save the combined DataFrame
# combined_file = os.path.join(output_relative_dir, 'all_properties_combined.csv')
# combined_df.to_csv(combined_file, index=False)
# print(f"Combined file saved at {combined_file}")

# # Delete the temporary files and folder
# for file in temp_files:
#     os.remove(file)
# os.rmdir(temp_folder)

# print(f"Temporary files and folder deleted.")


Visiting https://www.domain.com.au/rent/?ptype=apartment&bedrooms=1&postcode=3206&sort=dateupdated-desc&page=1


Processing Part 1, Postcode 3206: 100%|███████████████████████████████████████████████████| 9/9 [00:10<00:00,  1.13s/it]


Visiting https://www.domain.com.au/rent/?ptype=apartment&bedrooms=2&postcode=3206&sort=dateupdated-desc&page=1


Processing Part 1, Postcode 3206: 100%|███████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.12s/it]


Visiting https://www.domain.com.au/rent/?ptype=apartment&bedrooms=3&postcode=3206&sort=dateupdated-desc&page=1


...

Visiting https://www.domain.com.au/rent/?ptype=house&bedrooms=3&postcode=3691&sort=dateupdated-desc&page=1



Processing Part 10, Postcode 3691: 100%|██████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.21s/it]


Visiting https://www.domain.com.au/rent/?ptype=house&bedrooms=4&postcode=3691&sort=dateupdated-desc&page=1


Processing Part 10, Postcode 3691: 100%|██████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.12s/it]

Completed processing Part 10/10. Data saved to ../../data/raw/domain/temp_parts/property_part_10.csv
Combined file saved at ../../data/raw/domain/all_properties_combined.csv
Temporary files and folder deleted.


In [5]:
# download past data
url = "https://www.dffh.vic.gov.au/moving-annual-rent-suburb-december-quarter-2023-excel"

output_directory = "../../data/raw/domain/"
file_name = "past_data.xlsx"
output_path = os.path.join(output_directory, file_name)

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Write the content of the response to the specified path
    with open(output_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved as {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded successfully and saved as ../../data/raw/domain/past_data.xlsx
